Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/responsible-ai/auto-ml-regresion-responsibleai/auto-ml-regresion-responsibleai.png)

# Automated Machine Learning
_**Regression with Aml Compute**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Results](#Results)
1. [Test](#Test)


## Introduction
In this example we use the Hardware Performance Dataset to showcase how you can use AutoML for a simple regression problem. The regression goal is to predict the performance of certain combinations of hardware parts.
After training AutoML models for this regression data set, we show how you can compute model explanations on your remote compute using a sample explainer script.

If you are using an Azure Machine Learning Compute Instance, you are all set.  Otherwise, go through the [configuration](../../../configuration.ipynb)  notebook first if you haven't already to establish your connection to the AzureML Workspace. 

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Instantiate AutoMLConfig with FeaturizationConfig for customization.
3. Train the model using remote compute.
4. Explore the results and featurization transparency options.
5. Setup remote compute for computing the model explanations for a given AutoML model.
6. Start an AzureML experiment on your remote compute.
7. Submit model analysis, explain runs and counterfactual runs for a specific AutoML model.
8. Download the feature importance for raw features and visualize the explanations for raw features on azure portal. 
10. Download counterfactual examples and view them in the notebook.


## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
import azureml.dataprep as dprep
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.39.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
ws = Workspace.from_config()

# Choose a name for the experiment.
experiment_name = 'automl-regression-rai'
experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you create `AmlCompute` as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "hardware-rai"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### Setup Training and Test Data for AutoML experiment

Load the hardware dataset from a csv file containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. Next, we'll split the data using random_split and extract the training data for the model.  We also register the datasets in your workspace using a name so that these datasets may be accessed from the remote compute.

In [ ]:
data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/machineData.csv'

dataset = Dataset.Tabular.from_delimited_files(data)

# Split the dataset into train and test datasets
train_data, test_data = dataset.random_split(percentage=0.8, seed=223)

# Drop ModelName
train_data = train_data.drop_columns(['ModelName'])
test_data = test_data.drop_columns(['ModelName'])

# Register the train dataset with your workspace
train_data.register(workspace = ws, name = 'rai_machine_train_dataset',
                    description = 'hardware performance training data',
                    create_new_version=True)

# Register the test dataset with your workspace
test_data.register(workspace = ws, name = 'rai_machine_test_dataset', description = 'hardware performance test data', create_new_version=True)

label ="ERP"

train_data.to_pandas_dataframe().head()

## Train

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification, regression or forecasting|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**experiment_timeout_hours**| Maximum amount of time in hours that all iterations combined can take before the experiment terminates.|
|**enable_early_stopping**| Flag to enble early termination if the score is not improving in the short term.|
|**featurization**| 'auto' / 'off' / FeaturizationConfig Indicator for whether featurization step should be done automatically or not, or whether customized featurization should be used. Setting this enables AutoML to perform featurization on the input to handle *missing data*, and to perform some common *feature extraction*. Note: If the input data is sparse, featurization cannot be turned on.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|(sparse) array-like, shape = [n_samples, n_features]|
|**label_column_name**|(sparse) array-like, shape = [n_samples, ], targets values.|

## Customization

Supported customization includes:

1. Column purpose update: Override feature type for the specified column.
2. Transformer parameter update: Update parameters for the specified transformer. Currently supports Imputer and HashOneHotEncoder.
3. Drop columns: Columns to drop from being featurized.
4. Block transformers: Allow/Block transformers to be used on featurization process.

Create FeaturizationConfig object using API calls

In [ ]:
featurization_config = FeaturizationConfig()
featurization_config.blocked_transformers = ['LabelEncoder']
#featurization_config.drop_columns = ['MMIN']
featurization_config.add_column_purpose('MYCT', 'Numeric')
featurization_config.add_column_purpose('VendorName', 'CategoricalHash')
#default strategy mean, add transformer param for for 3 columns
featurization_config.add_transformer_params('Imputer', ['CACH'], {"strategy": "median"})
featurization_config.add_transformer_params('Imputer', ['CHMIN'], {"strategy": "median"})
featurization_config.add_transformer_params('Imputer', ['PRP'], {"strategy": "most_frequent"})
#featurization_config.add_transformer_params('HashOneHotEncoder', [], {"number_of_bits": 3})

In [ ]:
automl_settings = {
    "enable_early_stopping": True, 
    "experiment_timeout_hours" : 0.25,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'normalized_root_mean_squared_error',
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             featurization=featurization_config,
                             training_data = train_data,
                             label_column_name = label,
                             **automl_settings
                            )

Call the `submit` method on the experiment object and pass the `AutoMLConfig`. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.
In this example, we specify `show_output=False` to suppress output for each iteration.  You can monitor the run by clicking on the link in the output.

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)

Run the following cell to access previous runs. Uncomment the cell below and update the run_id.

In [ ]:
#from azureml.train.automl.run import AutoMLRun
#remote_run = AutoMLRun(experiment=experiment, run_id='AutoML_1723d4fe-c33d-41f7-83ad-c010215583b0')
#remote_run

In [ ]:
remote_run.wait_for_completion(wait_post_processing=True)

## Generating Responsible AI insights for AutoML model
This section will walk you through the workflow to compute Responsible AI insights like model explanations and counterfactual examples using model analysis workflow for an AutoML model on your remote compute.

### Retrieve any AutoML Model for explanations

Below we select an AutoML pipeline from our iterations. The `get_best_child` method returns the a AutoML run with the best score for the specified metric

In [ ]:
automl_run = remote_run.get_best_child(metric='mean_absolute_error')

### Setup model analysis on the remote compute
The following section provides details on how to setup an AzureML experiment to run model analysis for an AutoML model on your remote compute.

#### Create conda configuration for model analysis and explanations runs from automl_run object.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfiguration object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target

# specify CondaDependencies obj
conda_run_config.environment = automl_run.get_environment()

Register the AutoML model and create a `PickleModelLoader` for the model analysis so that the model analysis can instantiate the model downloaded from AzureML.

In [ ]:
from azureml.core import Model
from azureml.responsibleai.common.pickle_model_loader import PickleModelLoader
from azureml.responsibleai.tools.model_analysis.model_analysis_config import ModelAnalysisConfig
from azureml.responsibleai.tools.model_analysis.explain_config import ExplainConfig
from azureml.automl.core.shared.constants import MODEL_PATH

automl_run.download_file(name=MODEL_PATH, output_file_path='model.pkl')

model = automl_run.register_model(model_name='automl_rai', 
                                  model_path='outputs/model.pkl')

model_loader = PickleModelLoader('model.pkl')

Construct the list of the feature column names by dropping the name of the label column from the list of all column names.

In [ ]:
X_column_names = train_data.to_pandas_dataframe().columns.values
X_column_names = X_column_names[X_column_names!=label]
X_column_names

Get the train and test dataset for the model analysis.

In [ ]:
train_dataset = Dataset.get_by_name(workspace=ws, name='rai_machine_train_dataset')
test_dataset = Dataset.get_by_name(workspace=ws, name='rai_machine_test_dataset')

In the `ModelAnalysisConfig` below, `confidential_datastore_name` is the name of the datastore where the analyses will be uploaded.  This example uses the default data store because the dataset is also in the default datastore.  If you have confidential data in the dataset, you should specify a different data store as the `confidential_datastore_name` because analysis makes a copy of the data in this data store.

In [ ]:
categorical_features = ['VendorName']

model_analysis_config = ModelAnalysisConfig(
    title="Model analysis",
    model=model,
    model_type='regression',
    model_loader=model_loader,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    X_column_names=X_column_names,
    target_column_name=label,
    confidential_datastore_name=ws.get_default_datastore().name,
    run_configuration=conda_run_config,
    categorical_column_names=categorical_features
)

### Run model analysis

The model analysis run takes a snapshot of the data in preparation for model explanation, error analysis, causal and counterfactual.
The model analysis run is the parent run for the model explanation, error analysis, causal and counterfactual runs.
In this example we will just generate an explanation and counterfactuals, but causal and error analyses may be performed as well.

In [ ]:
model_analysis_run = experiment.submit(model_analysis_config)
model_analysis_run.wait_for_completion(raise_on_error=True, wait_post_processing=True)

## Compute explanations

Run model explanation based on the model analysis.
The explanation run is a child run of the model analysis run.
In the future, the `add_request` method will allow extra parameters to configure the explanation generated.

In [ ]:
explain_config = ExplainConfig(model_analysis_run, conda_run_config)
explain_config.add_request()
explain_run = model_analysis_run.submit_child(explain_config)
explain_run.wait_for_completion(raise_on_error=True, wait_post_processing=True)

The `explanation_manager.list` method below returns a list of metadata dictionaries for each explain run.  In this case, there is a single explain run.  So, the list contains a single dictionary.

In [ ]:
explanations = model_analysis_run.explanation_manager.list()
explanation = explanations[0]

### Feature importance  and  visualizing explanation dashboard
In this section we describe how you can download the explanation results from the explanations experiment and visualize the feature importance for your AutoML model on the azure portal.

In [ ]:
feature_explanations = model_analysis_run.explanation_manager.download_by_id(explanation['id'])
print(feature_explanations.get_feature_importance_dict())
print("You can visualize the explanations for your features under the 'Explanations (preview)' tab in the explain run at:-\n" + explain_run.get_portal_url())

## Generate counterfactual examples

Generate counterfactuals for all the samples in the `test_dataset` based on the model analysis.
The counterfactual run is a child run of the model analysis run.
In the future, the `add_request` method will allow extra parameters to configure the counterfactuals generated.

In [ ]:
from azureml.responsibleai.tools.model_analysis.counterfactual_config import CounterfactualConfig

cf_config = CounterfactualConfig(model_analysis_run, conda_run_config)
cf_config.add_request(total_CFs=10, desired_range=[10, 300])
cf_run = model_analysis_run.submit_child(cf_config)
cf_run.wait_for_completion(raise_on_error=True, wait_post_processing=True)

### Downloading counterfactual examples
The `counterfactual_manager.list` method below returns a list of metadata dictionaries for each counterfactual run.  In this case, there is a single counterfactual run.  So, the list contains a single dictionary.

The `download_by_id()` method available in the `counterfactual_manager` can be used to download the counterfactual examples.

In [ ]:
cf_meta = model_analysis_run.counterfactual_manager.list()
counterfactual_object = model_analysis_run.counterfactual_manager.download_by_id(cf_meta[0]['id'])

### Visualizing the generated counterfactuals
You can use `visualize_as_dataframe()` method to view the generated counterfactual examples for the samples in `test_dataset`.

In [ ]:
counterfactual_object.visualize_as_dataframe(show_only_changes=True)

### Visualize counterfactual feature importance

In [ ]:
counterfactual_object.summary_importance